# DBM Mask R-CNN Code

In [ ]:
!pip uninstall keras-nightly
!pip install h5py==2.10.0

     |████████████████████████████████| 2.9 MB 4.0 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires h5py~=3.1.0, but you have h5py 2.10.0 which is incompatible.


### Importing the required libraries for Weights & Biases integration and for Mask R-CNN code

In [ ]:
import os
import numpy as np
!pip install tensorflow==1.15.0rc2
import tensorflow as tf
!pip install keras==2.2.5

     |████████████████████████████████| 412.3 MB 23 kB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 3.8 MB 48.5 MB/s 
     |████████████████████████████████| 503 kB 42.7 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=0add66e5740a3d77412648b96f045d1a694ebfdd144355ac6695778cfc47ab09
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

### Installing and logging into Weights & Biases account

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Mounting drive to upload files from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


### Uploading DBM dataset file (with images and XML files) and visualize, utils & model Python files

In [ ]:
!unzip "//gdrive//My Drive//DBM Mask RCNN//Updated New Data.zip"
!cp "/gdrive/My Drive/DBM Mask RCNN/visualize_saveimage_orderedfilename.py" "visualize.py"
!cp "/gdrive/My Drive/DBM Mask RCNN/utils.py" "utils.py"
!cp "/gdrive/My Drive/DBM Mask RCNN/model_savetodrive.py" "model.py"

Archive:  //gdrive//My Drive//DBM Mask RCNN//Updated New Data.zip
   creating: Updated New Data/annots/
  inflating: Updated New Data/annots/1.xml  
  inflating: Updated New Data/annots/10.xml  
  inflating: Updated New Data/annots/100.xml  
  inflating: Updated New Data/annots/101.xml  
  inflating: Updated New Data/annots/102.xml  
  inflating: Updated New Data/annots/103.xml  
  inflating: Updated New Data/annots/104.xml  
  inflating: Updated New Data/annots/105.xml  
  inflating: Updated New Data/annots/106.xml  
  inflating: Updated New Data/annots/107.xml  
  inflating: Updated New Data/annots/108.xml  
  inflating: Updated New Data/annots/109.xml  
  inflating: Updated New Data/annots/11.xml  
  inflating: Updated New Data/annots/110.xml  
  inflating: Updated New Data/annots/111.xml  
  inflating: Updated New Data/annots/112.xml  
  inflating: Updated New Data/annots/113.xml  
  inflating: Updated New Data/annots/114.xml  
  inflating: Updated New Data/annots/115.xml  
  infla

### Importing the required libraries for Mask R-CNN code

In [ ]:
import xml.etree
from numpy import zeros, asarray
from random import sample
import uuid

!pip install mrcnn
import mrcnn
import mrcnn.config
import visualize
import utils
import model

     |████████████████████████████████| 51 kB 246 kB/s 
  Created wheel for mrcnn: filename=mrcnn-0.2-py3-none-any.whl size=54930 sha256=d1eda7437c9840c1f9e4abeffe86a7e3470a2bb4b4758a19980f76be973f9bb7
  Stored in directory: /root/.cache/pip/wheels/1d/94/0d/03ff96abc43d2d6c8299a92cbb4eced2a1eda3ca7911c19427
Successfully built mrcnn


Using TensorFlow backend.
wandb: WARNING Keras version 2.2.5 is not fully supported. Required keras >= 2.4.0


### DBM Configuration

In [ ]:
class DBMConfig(mrcnn.config.Config): 
    NAME = "DBM_cfg"

    BACKBONE = "resnet50" # can use resnet101 as well

    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    
    NUM_CLASSES = 2 # (background/non-DBM + DBM)

    STEPS_PER_EPOCH = 517 # number of training images (738 total images - 221 validation images)

    PRE_NMS_LIMIT = 6000

    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)
    BACKBONE_STRIDES = [4, 8, 16, 32, 64]

    IMAGE_MAX_DIM = 1024
    IMAGE_MIN_DIM = 1024 # resizes all images to 1024x1024px

    MAX_GT_INSTANCES = 1000
  
DBM_config = DBMConfig()
DBM_config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

### DBM Dataset

In [ ]:
randimage_id = (sample(range(1,739),221))  # 221 numbers are randomly generated to pass its respective 221 images to the validation set
print(randimage_id)
random_uuid = str(uuid.uuid4())
print("val_set_images_%s.txt" % random_uuid)
with open("/gdrive/My Drive/DBM Mask RCNN/val_set_images_%s.txt" % random_uuid, "w") as file: # the 221 validation set images' filenames are saved onto a text document and into Google Drive, to be loaded up later when continuing interrupted training or testing weights
  for num in randimage_id:
    file.write("%i\n" % num)

[95, 411, 386, 274, 673, 357, 469, 465, 488, 195, 33, 415, 268, 736, 316, 540, 328, 369, 591, 126, 637, 168, 371, 735, 448, 6, 64, 99, 666, 687, 732, 284, 219, 102, 327, 39, 572, 44, 163, 358, 438, 569, 229, 635, 536, 575, 461, 143, 255, 59, 464, 156, 344, 664, 728, 389, 379, 674, 682, 393, 459, 670, 299, 639, 60, 429, 718, 439, 685, 351, 187, 179, 40, 444, 526, 402, 705, 75, 643, 443, 312, 450, 49, 198, 510, 512, 675, 43, 285, 467, 644, 584, 738, 522, 560, 67, 77, 89, 125, 676, 482, 456, 564, 544, 194, 84, 507, 594, 240, 196, 225, 449, 90, 383, 462, 636, 111, 548, 408, 490, 626, 491, 72, 451, 135, 134, 343, 12, 311, 586, 396, 511, 422, 314, 581, 25, 164, 224, 603, 201, 239, 487, 632, 22, 108, 21, 37, 189, 616, 374, 71, 50, 412, 81, 706, 218, 290, 571, 484, 142, 495, 516, 54, 118, 342, 263, 38, 329, 146, 519, 62, 346, 160, 333, 177, 56, 296, 370, 193, 180, 289, 470, 209, 430, 330, 213, 606, 372, 726, 381, 297, 658, 305, 363, 435, 696, 166, 57, 426, 628, 117, 719, 315, 457, 243, 481, 56

In [ ]:
class DBMDataset(utils.Dataset):
  
  def load_dataset(self, dataset_dir, is_train = True):
    self.add_class("dataset", 1, "DBM")
    
    images_dir = dataset_dir + '/images/'
    annotations_dir = dataset_dir + '/annots/'
    
    for filename in os.listdir(images_dir):
      image_id = filename.split('.')[0] # removes .jpg/.jpeg from the name of the file

      # skip the 221 images supposed to be in the validaion set if we are building the train set
      if is_train and int(image_id) in randimage_id:
        continue

      # skip the images apart from the 221 images supposed to be in the validaion set if we are building the validation set
      if not is_train and int(image_id) not in randimage_id:
        continue
      
      img_path = images_dir + filename
      ann_path = annotations_dir + image_id + '.xml'
      
      self.add_image('dataset', image_id = image_id, path = img_path, annotation = ann_path)

  def extract_boxes(self, filename):
    tree = xml.etree.ElementTree.parse(filename)
    root = tree.getroot()
    
    boxes = list()
    for box in root.findall('.//bndbox'):
      xmin = int(box.find('xmin').text)
      ymin = int(box.find('ymin').text)
      xmax = int(box.find('xmax').text)
      ymax = int(box.find('ymax').text)
      coors = [xmin, ymin, xmax, ymax]
      boxes.append(coors)
    
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height
    
  def load_mask(self, image_id):
    info = self.image_info[image_id]
    path = info['annotation']
    boxes, w, h = self.extract_boxes(path)
    masks = zeros([h, w, len(boxes)], dtype = 'uint8')

    class_ids = list()
    for i in range(len(boxes)):
      box = boxes[i]
      row_s, row_e = box[1], box[3]
      col_s, col_e = box[0], box[2]
      masks[row_s:row_e, col_s:col_e, i] = 1
      class_ids.append(self.class_names.index('DBM'))
    return masks, asarray(class_ids, dtype = 'int32')

### Prepare training and validation sets

In [ ]:
train_set = DBMDataset()
train_set.load_dataset(dataset_dir = 'Updated New Data', is_train = True)
train_set.prepare()

valid_dataset = DBMDataset()
valid_dataset.load_dataset(dataset_dir = 'Updated New Data', is_train = False)
valid_dataset.prepare()

### Initializing and configuring Weights & Biases with the training run

In [ ]:
run = wandb.init(project='mask-rcnn-testing')

wandb: Currently logged in as: ronithsaju (use `wandb login --relogin` to force relogin)


### Initialize Mask R-CNN model for training

In [ ]:
DBM_model = model.MaskRCNN(mode = 'training', 
                           model_dir = './', 
                           config = DBM_config)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


### Load pre-trained ImageNet weights (excluding last few layers) [to use if training model with layers = "heads"]

In [ ]:
'''
DBM_model.load_weights(filepath = '/gdrive/My Drive/DBM Mask RCNN/mask_rcnn_coco.h5', 
                       by_name = True, 
                       exclude = ["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
'''

'\nDBM_model.load_weights(filepath = \'/gdrive/My Drive/DBM Mask RCNN/mask_rcnn_coco.h5\', \n                       by_name = True, \n                       exclude = ["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])\n'

In [ ]:
'''
weights_path = DBM_model.get_imagenet_weights()

DBM_model.keras_model.load_weights(weights_path, 
                                   by_name=True, 
                                   skip_mismatch=True)
'''

'\nweights_path = DBM_model.get_imagenet_weights()\n\nDBM_model.keras_model.load_weights(weights_path, \n                                   by_name=True, \n                                   skip_mismatch=True)\n'

### Training the model

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
DBM_model.train(train_dataset = train_set, 
                val_dataset = valid_dataset, 
                learning_rate = DBM_config.LEARNING_RATE, 
                epochs = 10, 
                layers = 'all', 
                custom_callbacks = [WandbCallback()])


Starting at epoch 0. LR=0.001

Checkpoint Path: /gdrive/My Drive/DBM Mask RCNN/Config Files/dbm_cfg20210826T0600/mask_rcnn_dbm_cfg_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         

/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multi


Epoch 1/10
123/517 [======>.......................] - ETA: 14:42 - loss: 3.6499 - rpn_class_loss: 0.8597 - rpn_bbox_loss: 1.7287 - mrcnn_class_loss: 0.1601 - mrcnn_bbox_loss: 0.5775 - mrcnn_mask_loss: 0.3239

/usr/local/lib/python3.7/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


517/517 [==============================] - 1083s 2s/step - loss: 2.2100 - rpn_class_loss: 0.3112 - rpn_bbox_loss: 0.8104 - mrcnn_class_loss: 0.2153 - mrcnn_bbox_loss: 0.4675 - mrcnn_mask_loss: 0.4055 - val_loss: 1.6130 - val_rpn_class_loss: 0.0615 - val_rpn_bbox_loss: 0.5627 - val_mrcnn_class_loss: 0.2457 - val_mrcnn_bbox_loss: 0.3709 - val_mrcnn_mask_loss: 0.3721


wandb: ERROR Can't save model, h5py returned error: can't pickle _thread.RLock objects


Epoch 2/10
517/517 [==============================] - 761s 1s/step - loss: 1.5005 - rpn_class_loss: 0.0735 - rpn_bbox_loss: 0.3947 - mrcnn_class_loss: 0.2532 - mrcnn_bbox_loss: 0.3729 - mrcnn_mask_loss: 0.4062 - val_loss: 1.5242 - val_rpn_class_loss: 0.0959 - val_rpn_bbox_loss: 0.3963 - val_mrcnn_class_loss: 0.2263 - val_mrcnn_bbox_loss: 0.3839 - val_mrcnn_mask_loss: 0.4219
Epoch 3/10
517/517 [==============================] - 1073s 2s/step - loss: 1.3427 - rpn_class_loss: 0.0586 - rpn_bbox_loss: 0.3182 - mrcnn_class_loss: 0.2412 - mrcnn_bbox_loss: 0.3368 - mrcnn_mask_loss: 0.3879 - val_loss: 1.3449 - val_rpn_class_loss: 0.0660 - val_rpn_bbox_loss: 0.3319 - val_mrcnn_class_loss: 0.1929 - val_mrcnn_bbox_loss: 0.3471 - val_mrcnn_mask_loss: 0.4069
Epoch 4/10
517/517 [==============================] - 1028s 2s/step - loss: 1.2413 - rpn_class_loss: 0.0449 - rpn_bbox_loss: 0.2787 - mrcnn_class_loss: 0.2374 - mrcnn_bbox_loss: 0.3059 - mrcnn_mask_loss: 0.3744 - val_loss: 1.3775 - val_rpn_class

### Saving the trained weights generated

In [ ]:
model_path = 'DBM_mask_rcnn.h5'
DBM_model.keras_model.save_weights(model_path)

### Loading the model

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

# Loading the model in the inference mode
DBM_model = model.MaskRCNN(mode = "inference", config = DBM_config, model_dir = './')
# loading the trained weights
DBM_model.load_weights(model_path, by_name=True)


Instructions for updating:
Use `tf.cast` instead.


### Displaying and saving all the results into DBM Model Results folder (inside DBM Mask RCNN folder), and calculating mAP

In [ ]:
#os.makedirs('/gdrive/My Drive/DBM Mask RCNN/DBM Model Results/change this name') 

# uncomment the above code and change the "save_image" parameter below in the visualize.display_differences() & visualize.display_instances() functions to True
# if you want to save all the annotation results images to the Drive folder named 'change this name'

In [ ]:
image_id = 0
APs = []
mPrecision = []
mRecall = []

TPs = 0
FPs = 0
FNs = 0
relativeTPs = []
relativeFPs = []
relativeFNs = []

total_images_GT = 0
total_images_noGT = 0

total_FP_no_DBM = 0

for image_id in valid_dataset.image_ids:
  image, image_meta, gt_class_id, gt_bbox, gt_mask = model.load_image_gt(valid_dataset, DBM_config, image_id, use_mini_mask=False)

  print("===================================================================================================================")
  print("image_id: ", image_id)
  info = valid_dataset.image_info[image_id]
  results = DBM_model.detect([image], verbose=0) # the Mask R-CNN model predicts on the DBM trap image here
  r = results[0]

  if len(gt_bbox) != 0: # meaning images with 1 or more DBMs in it

    '''
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                valid_dataset.class_names, r['scores'], 
                                title=str(info["id"]), show_mask = False, save_image = False, filename = int(info["id"]))
    
    visualize.display_instances(image, gt_bbox, gt_mask, gt_class_id, 
                                valid_dataset.class_names, 
                                title=str(info["id"]), show_mask = False, save_image = False, filename = int(info["id"]))
    '''

    visualize.display_differences(image, gt_bbox, gt_class_id, gt_mask, 
                                  r['rois'], r['class_ids'], r['scores'], r['masks'], 
                                  valid_dataset.class_names, title=str(info["id"]), 
                                  ax=None, show_mask=False, show_box=True, 
                                  iou_threshold=0.5, score_threshold=0.5, save_image = False, filename = int(info["id"])) 
    
    # the model-produced annotations are visualized using the above function, with the green boxes being GTs and the red boxes being the model's predictions for DBMs

    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'], iou_threshold=0.5)

    APs.append(AP)
    #visualize.plot_precision_recall(AP, precisions, recalls)
    print("AP: ", AP)
    print("precision value: ", precisions[-2])
    mPrecision.append(precisions[-2])

    recall, positive_ids = utils.compute_recall(r['rois'], gt_bbox, 0.5)
    print("recall value: ", recall)
    mRecall.append(recall)

    print("current mAP: ", np.mean(APs))
    print("current Mean Precision: ", np.mean(mPrecision))
    print("current Mean Recall: ", np.mean(mRecall))

    gt_match, pred_match, overlaps = utils.compute_matches(gt_bbox, gt_class_id, gt_mask, 
                                                           r['rois'], r['class_ids'], r['scores'], r['masks'], 
                                                           iou_threshold=0.5)
    
    TP = np.sum(pred_match > -1)
    print("TP: ", TP)
    TPs += TP

    FP = np.sum(pred_match == -1)
    print("FP: ", FP)
    FPs += FP

    FN = len(gt_match) - np.sum(pred_match > -1)
    print("FN: ", FN)
    FNs += FN


    if(len(pred_match) != 0):
      relativeTP = TP/len(pred_match)
      print("relativeTP: ", relativeTP)
      relativeTPs.append(relativeTP)

      relativeFP = FP/len(pred_match)
      print("relativeFP: ", relativeFP)
      relativeFPs.append(relativeFP)

    else:
      relativeTP = TP
      print("relativeTP: ", relativeTP)
      relativeTPs.append(relativeTP)

      relativeFP = FP
      print("relativeFP: ", relativeFP)
      relativeFPs.append(relativeFP)

    relativeFN = FN/len(gt_match)
    print("relativeFN: ", relativeFN)
    relativeFNs.append(relativeFN)

    total_images_GT += 1

    #utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'], iou_thresholds=None, verbose=1)

  else: # meaning images with 0 DBMs in it
    print("No DBMs present in image %s" % str(image_id))
    total_images_noGT += 1

    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                valid_dataset.class_names, r['scores'], 
                                title=str(info["id"]), show_mask = False, save_image = False, filename = int(info["id"]))
    
    # the model-produced annotations are visualized using the above function, with the colourful boxes being the model's predictions for DBMs on images with 0 GTS

    print("Number of wrong predictions:", len(r['rois']))
    total_FP_no_DBM += len(r['rois'])

In [ ]:
print("mAP:", np.mean(APs))
print("Mean Precision:", np.mean(mPrecision))
print("Mean Recall:", np.mean(mRecall))

print("Total TP:", TPs)
print("Total FP:", FPs)
print("Total FN:", FNs)

print("Average Relative TP:", np.mean(relativeTPs))
print("Average Relative FP:", np.mean(relativeFPs))
print("Average Relative FN:", np.mean(relativeFNs))

print("Total FP no DBM:", total_FP_no_DBM)
print("Total num of images with no DBMs:", total_images_noGT)
total_FP_combined = FPs + total_FP_no_DBM
print("Total Combined FP:", total_FP_combined)

mAP: 0.6447744922590365
Mean Precision: 0.38190158286062253
Mean Recall: 0.8454958281470923
Total TP: 1474
Total FP: 2097
Total FN: 556
Average Relative TP: 0.38190158286062253
Average Relative FP: 0.6180984171393774
Average Relative FN: 0.17527685595273787
Total FP no DBM: 863
Total num of images with no DBMs: 62
Total Combined FP: 2960


In [ ]:
wandb.log({"mAP": np.mean(APs)})
wandb.log({"Mean Precision": np.mean(mPrecision)})
wandb.log({"mAR": np.mean(mRecall)})

wandb.log({"Total TP": TPs})
wandb.log({"Total FP": FPs})
wandb.log({"Total FN": FNs})

wandb.log({"Average Relative TP": np.mean(relativeTPs)})
wandb.log({"Average Relative FP": np.mean(relativeFPs)})
wandb.log({"Average Relative FN": np.mean(relativeFNs)})

wandb.log({"Total FP no DBMs": total_FP_no_DBM})
wandb.log({"Total num of images with no DBMs": total_images_noGT})
wandb.log({"Total Combined FP": total_FP_combined})

wandb.finish()